In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/APS360-Project-Data/archive.zip  -d /content/drive/MyDrive/APS360-Project-Data

In [ ]:
from PIL import Image
import os
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
!ls /content/drive/MyDrive/APS360-Project-Data/images | wc -l

433


In [ ]:
dire_image_resized = '/content/drive/MyDrive/APS360-Project-Data/images_resized_test'
if(os.path.isdir(dire_image_resized) == False):
  os.mkdir(dire_image_resized)

dire_labels = '/content/drive/MyDrive/APS360-Project-Data/labels_test'
if(os.path.isdir(dire_labels) == False):
  os.mkdir(dire_labels)

In [ ]:
#resize images and format labels 
index = 0

new_width = 512
new_height = 512

for i in range(433):
  #parse xml files 
  path = '/content/drive/MyDrive/APS360-Project-Data/annotations/Cars{}.xml'.format(i)
  with open(path) as f:
    xml_file = f.read()

  bs_data = BeautifulSoup(xml_file, 'xml')
  box_xmin = bs_data.find_all('xmin')

  #skip images with multiple LPs
  if len(box_xmin) > 1:
    continue

  #read width and height of the image
  image_width = bs_data.find_all('width')[0].string
  image_height = bs_data.find_all('height')[0].string

  #read bounding box coordinates from xml file
  box_xmin = bs_data.find_all('xmin')[0].string
  box_ymin = bs_data.find_all('ymin')[0].string
  box_xmax = bs_data.find_all('xmax')[0].string
  box_ymax = bs_data.find_all('ymax')[0].string

  box_xmin_resized = new_width / float(image_width) * float(box_xmin)
  box_ymin_resized = new_height / float(image_height) * float(box_ymin)
  box_xmax_resized = new_width / float(image_width) * float(box_xmax)
  box_ymax_resized = new_height / float(image_height) * float(box_ymax)

  #normalize center coordinates and width, height to 0~1
  center_x = 0.5 * (box_xmin_resized + box_xmax_resized) / new_width
  center_y = 0.5 * (box_ymin_resized + box_ymax_resized) / new_height

  width = (box_xmax_resized - box_xmin_resized) / new_width
  height = (box_ymax_resized - box_ymin_resized) / new_height

  #box_coordinates = [box_xmin_resized, box_ymin_resized, box_xmax_resized, box_ymax_resized]
  box_coordinates = [center_x, center_y, width, height]

  np.savetxt('/content/drive/MyDrive/APS360-Project-Data/dataset_from_Kaggle/labels/label{}.txt'.format(index), box_coordinates, fmt='%f', newline=' ')

  #resize image
  image = Image.open(r'/content/drive/MyDrive/APS360-Project-Data/images/Cars{}.png'.format(i))
  image = image.convert('RGB')

  image = image.resize((512, 512))
  

  image.save('/content/drive/MyDrive/APS360-Project-Data/dataset_from_Kaggle/images/Cars{}_resized_old_{}.jpg'.format(index, i))
  image.show()

  index += 1
  



In [ ]:
!ls /content/drive/MyDrive/APS360-Project-Data/dataset_from_Kaggle/images | wc -l

409


In [ ]:
!ls /content/drive/MyDrive/APS360-Project-Data/dataset_from_Kaggle/labels | wc -l

409


In [ ]:
!ls /content/drive/MyDrive/APS360-Project-Data/CCPD-base-partial > /content/drive/MyDrive/APS360-Project-Data/filename.txt

/bin/bash: /content/drive/MyDrive/APS360-Project-Data/filename.txt: No such file or directory


In [ ]:
dire_CCPD_resized = '/content/drive/MyDrive/APS360-Project-Data/CCPD_resized'
if(os.path.isdir(dire_CCPD_resized) == False):
  os.mkdir(dire_CCPD_resized)

dire_CCPD_labels = '/content/drive/MyDrive/APS360-Project-Data/CCPD_labels'
if(os.path.isdir(dire_CCPD_labels) == False):
  os.mkdir(dire_CCPD_labels)



In [ ]:
f = open('/content/drive/MyDrive/APS360-Project-Data/test.txt', 'r')
line = f.readline()

while len(line) > 1:
  print(line)
  line = f.readline()

In [ ]:
!ls /content/drive/MyDrive/APS360-Project-Data/CCPD-base-partial | wc -l

3910


In [ ]:
!du -sh /content/drive/MyDrive/APS360-Project-Data/CCPD-base-partial

276M	/content/drive/MyDrive/APS360-Project-Data/CCPD-base-partial


In [ ]:
f = open('/content/drive/MyDrive/APS360-Project-Data/filename.txt', 'r')

line = f.readline()
resize_to = (256, 256)
count = 0

while len(line) > 1 and count < 5:
  line = line.rstrip('\n')
  path = os.path.join('/content/drive/MyDrive/APS360-Project-Data/CCPD-base-partial', line)
  image = Image.open(path)

  image = image.convert('RGB')

  image = image.resize(resize_to)

  image.save('/content/drive/MyDrive/APS360-Project-Data/CCPD_resized_test/resized{}.jpg'.format(count))

  image.show()
  count += 1
  line = f.readline()

f.close()

In [ ]:
f = open('/content/drive/MyDrive/APS360-Project-Data/filename.txt', 'r')

line = f.readline()
#print(line)
#line = line[:-1]
#print(line)

resize_to = (512, 512)

count = 0
while len(line) > 1:
  if count % 10 <= 5:
    saving_path = '/content/drive/MyDrive/APS360-Project-Data/dataset_CCPD_resized/train'
  elif count % 10 <= 7:
    saving_path = '/content/drive/MyDrive/APS360-Project-Data/dataset_CCPD_resized/val'
  else:
    saving_path = '/content/drive/MyDrive/APS360-Project-Data/dataset_CCPD_resized/test'

  line = line.rstrip('\n')
  path = os.path.join('/content/drive/MyDrive/APS360-Project-Data/CCPD-base-partial', line)
  image = Image.open(path)
  
  old_width, old_height = image.size
  #print("old size: ", old_width, ' ', old_height)

  # if old_width != 720 and old_height != 1160:
  #   print("diff size: ")
  #   print("old size: ", old_width, ' ', old_height)
  #   break
  
  image = image.convert('RGB')

  image = image.resize(resize_to)

  #new_width, new_height = image.size
  #print("new size: ", new_width, ' ', new_height, '\n')

  
  image_saving_path = os.path.join(saving_path, 'images')
  image.save(os.path.join(image_saving_path, 'resized{}.jpg'.format(count)))

  image.show()
  
  # #save labels
  items = line.split('-')

  # coords = items[3].split('_')

  # coord1_x, coord1_y = coords[0].split('&')
  # coord2_x, coord2_y = coords[1].split('&')
  # coord3_x, coord3_y = coords[2].split('&')
  # coord4_x, coord4_y = coords[3].split('&')

  # bottom_right_x = new_width / float(old_width) * float(coord1_x)
  # bottom_right_y = new_height / float(old_height) * float(coord1_y)

  # bottom_left_x = new_width / float(old_width) * float(coord2_x)
  # bottom_left_y = new_height / float(old_height) * float(coord2_y)

  # top_left_x = new_width / float(old_width) * float(coord3_x)
  # top_left_y = new_height / float(old_height) * float(coord3_y)
  
  # top_right_x = new_width / float(old_width) * float(coord4_x)
  # top_right_y = new_height / float(old_height) * float(coord4_y)
  coords = items[2].split('_')
  box_xmin, box_ymin = coords[0].split('&')
  box_xmax, box_ymax = coords[1].split('&')

  #normalize center coordinates and width, height to 0~1
  center_x = 0.5 * (float(box_xmin) + float(box_xmax)) / old_width
  center_y = 0.5 * (float(box_ymin) + float(box_ymax)) / old_height

  width = (float(box_xmax) - float(box_xmin)) / old_width
  height = (float(box_ymax) - float(box_ymin)) / old_height

  #label_coordinates = [bottom_right_x, bottom_right_y, bottom_left_x, bottom_left_y, top_left_x, top_left_y, top_right_x, top_right_y]
  box_coordinates = [center_x, center_y, width, height]

  label_saving_path = os.path.join(saving_path, 'labels')
  
  np.savetxt(os.path.join(label_saving_path, 'label{}.txt'.format(count)), box_coordinates, fmt='%f', newline=' ')

  count += 1

  line = f.readline()

# print("All images have the same size")
# print("There are {} images".format(count))
f.close()
  



In [ ]:
!ls /content/drive/MyDrive/APS360-Project-Data/dataset_CCPD_resized/train/images | wc -l

2346


In [ ]:
!ls /content/drive/MyDrive/APS360-Project-Data/dataset_CCPD_resized/val/images | wc -l

782


In [ ]:
!ls /content/drive/MyDrive/APS360-Project-Data/dataset_CCPD_resized/test/images | wc -l

782


In [ ]:
!unzip /content/drive/MyDrive/APS360-Project-Data/nx9xbs4rgx-2.zip -d /content/drive/MyDrive/APS360-Project-Data

In [ ]:
dire_image_to_crop = '/content/drive/MyDrive/APS360-Project-Data/image_to_crop'
if(os.path.isdir(dire_image_to_crop) == False):
  os.mkdir(dire_image_to_crop)

In [ ]:
!mv /content/drive/MyDrive/APS360-Project-Data/images/*.JPG /content/drive/MyDrive/APS360-Project-Data/image_to_crop

In [ ]:
dir = '/content/drive/MyDrive/APS360-Project-Data/images'

dire_cropped_LP = '/content/drive/MyDrive/APS360-Project-Data/LP_cropped'
if(os.path.isdir(dire_cropped_LP) == False):
  os.mkdir(dire_cropped_LP)

for i in range(5):
  #crop LP from image
  image = Image.open(r'/content/drive/MyDrive/APS360-Project-Data/image_to_crop/cropped_parking_lot_{}.JPG'.format(i+1))
  image = image.convert('RGB')

  f = open('/content/drive/MyDrive/APS360-Project-Data/cropped_parking_lot_{}.txt'.format(i+1), 'r')